<a href="https://colab.research.google.com/github/mansueli/Supa-Migrate/blob/main/Amazon_RDS_to_Supabase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Supabase](https://raw.githubusercontent.com/supabase/supabase/master/packages/common/assets/images/supabase-logo-wordmark--light.svg)



#Amazon RDS to Supabase migration guide

In [ ]:
#@title Select the `Engine` of the RDS Database & install required resources. { display-mode: "form" }
engine = 'Postgres' #@param ["Postgres", "MySQL", "MS SQL","SQLite"]

if engine=='Postgres':
  !sudo sh -c 'echo "deb http://apt.postgresql.org/pub/repos/apt $(lsb_release -cs)-pgdg main" > /etc/apt/sources.list.d/pgdg.list'
  !wget --quiet -O - https://www.postgresql.org/media/keys/ACCC4CF8.asc | sudo apt-key add -
  !sudo apt-get update &>log
  !sudo apt-get -y install postgresql &>log
  !wget https://raw.githubusercontent.com/mansueli/Supa-Migrate/main/migrate_postgres_project.sh &>log
  !wget https://raw.githubusercontent.com/mansueli/Supa-Migrate/main/after.sql
  !wget https://raw.githubusercontent.com/mansueli/Supa-Migrate/main/before.sql
  print("Installed Postgres")
else:
  !wget https://github.com/dimitri/pgloader/archive/refs/tags/v3.6.9.zip &>log
  !apt-get install sbcl unzip libsqlite3-dev make curl gawk freetds-dev libzip-dev &>log
  !unzip v3.6.9.zip -d pgloader &>log
  !make -C ./pgloader/pgloader-3.6.9  &>log
  !./pgloader/pgloader-3.6.9/build/bin/pgloader --help
  print("Pgloader installed to migrated "+engine+ "to Supabase")

## Set the environment Variables:

In [5]:
#Source DB variables:
%env HOST=duplicate-of-workweek-prod-for-testing.c4ffsv6b9ash.eu-west-1.rds.amazonaws.com
%env USER=idevadmin
%env SOURCE_DB=postgres
%env PASSWORD=d9dGJhZgwB3kykQpqXGXeFYW9aU
%env PORT=5432
#Supabase variables:
%env SUPABASE_URL=https://ytlqeucmuaongxxdneal.supabase.co
%env SUPABASE_PASSWORD=d9dGJhZgwB3kykQpqXGXeFYW9aU
%env SUPABASE_PORT=5432


env: HOST=db.btqombjmkczzhqfjylvb.supabase.co
env: USER=postgres
env: SOURCE_DB=postgres
env: PASSWORD=5IqmDPoi0PpO6W4I
env: PORT=5432
env: SUPABASE_URL=https://kdniubdfnyrxlsyjsite.supabase.co
env: SUPABASE_PASSWORD=CDoIOoO3JjgxwkhK
env: SUPABASE_PORT=5432


In [ ]:
#@title #Running the Migration: { display-mode: "form" }
import os

supabase_url = os.environ['SUPABASE_URL']

if engine=='MySQL':
  !./pgloader/pgloader-3.6.9/build/bin/pgloader --before before.sql --after after.sql 'mysql://$USER:$PASSWORD@$HOST:$PORT/$SOURCE_DB' 'pgsql://postgres:$SUPABASE_PASSWORD@$SUPABASE_URL:$SUPABASE_PORT/postgres' #&>log
if engine=='MS SQL':
  !./pgloader/pgloader-3.6.9/build/bin/pgloader --before before.sql --after after.sql 'mssql://$USER:$PASSWORD@$HOST:$PORT/$SOURCE_DB' 'pgsql://postgres:$SUPABASE_PASSWORD@$SUPABASE_URL:$SUPABASE_PORT/postgres' #&>log
if engine=='SQLite':
  !./pgloader/pgloader-3.6.9/build/bin/pgloader --before before.sql --after after.sql 'sqllite://$USER:$PASSWORD@$HOST:$PORT/$SOURCE_DB' 'pgsql://postgres:$SUPABASE_PASSWORD@$SUPABASE_URL:$SUPABASE_PORT/postgres' #&>log
if engine=='Postgres':
  sender_db = os.environ['SOURCE_DB']
  sender_user = os.environ['USER']
  sender_host = os.environ['HOST']
  sender_pgpass = os.environ['PASSWORD']
  filedata = ''
  with open('migrate_postgres_project.sh', 'r') as file :
    filedata = file.read()
    filedata = filedata.replace('POSTGRES_ORIGIN_USERNAME', sender_user)
    filedata = filedata.replace('POSTGRES_ORIGIN_PASSWORD', sender_pgpass.replace("PGPASSWORD=",""))
    filedata = filedata.replace('POSTGRES_ORIGIN_DATABASE', sender_db)
    filedata = filedata.replace('POSTGRES_ORIGIN_HOST', sender_host)
    filedata = filedata.replace('SUPABASE_HOST', os.environ["SUPABASE_URL"].replace('https://','db.'))
    filedata = filedata.replace('SUPABASE_PASSWORD', os.environ['SUPABASE_PASSWORD'])

  with open('migrate_postgres_project.sh', 'w') as file:
    file.write(filedata)

  !bash ./migrate_postgres_project.sh #&>log
print("Migration completed")